# Notebook de Carga de Datos iniciales

Este notebook esta destinado a la carga inicial de los datos en los distintas base de datos

## Importación de librerias

Se importan las librerias necesarias para la carga de datos masiva.

In [15]:
# Rutas
from pathlib import Path

#Conexiones BD
from db_connections import client,db_neo4j,db_redis

#Funciones BD
from src import mongo,neo4j,utils,redis
from pymongo.errors import ConnectionFailure

#Procesar archivos/datos
from src.utils import procesar_csv
from datetime import datetime
import os
import pandas as pd


## Conexión con bases de datos

In [17]:
# =====================
# TEST DE CONEXIONES
# =====================

#Conecto con MongoDB
try:
    client.admin.command("ping")  # fuerza conexión al servidor
    print("✅ Conexión a MongoDB verificada.")
except ConnectionFailure as e:
    print(f"❌ Falló la conexión: {type(e).__name__} - {e}")

#Conecto con Neo4j
try:
    db_neo4j.verify_connectivity()
    print("✅ Conexión a Neo4j verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

#Conecto con Redis
try:
    db_redis.ping()
    print("✅ Conexión a Redis verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

✅ Conexión a MongoDB verificada.
✅ Conexión a Neo4j verificada.
✅ Conexión a Redis verificada.


## Limpieza previa

Esta sección elimina toda la información existente en las bases de datos utilizadas por el proyecto, dejándolas listas para una nueva carga o ejecución del proceso completo.
Las acciones que se realizan son las siguientes:

1. **`MongoDB`** → Se elimina la base de datos completa (clase) utilizando `drop_database()`.

2. **`Redis`** → Se vacía completamente la base mediante `flushdb()`.

3. **`Neo4j`** → Se eliminan todos los nodos y relaciones mediante la consulta `MATCH (n) DETACH DELETE n`.

In [18]:
# --- LIMPIAR DATOS EXISTENTES EN TODAS LAS BASES ---

print("\nLimpiando datos existentes...")

# --- MONGO ---
nombre_base = "clase"
client.drop_database(nombre_base)
print("   ✓ Base de datos Mongo eliminada")

# --- REDIS ---
db_redis.flushdb()
print("   ✓ Base de datos Redis limpiada")

# --- NEO4J ---
with db_neo4j.session() as session:
    session.run("MATCH (n) DETACH DELETE n")
print("   ✓ Nodos y relaciones de Neo4j eliminados")

print("\n Todos los datos fueron limpiados correctamente.")



Limpiando datos existentes...
   ✓ Base de datos Mongo eliminada
   ✓ Base de datos Redis limpiada
   ✓ Nodos y relaciones de Neo4j eliminados

 Todos los datos fueron limpiados correctamente.


## Carga de Datos

Bloque para la carga masiva de datos de las tres bases de datos

In [19]:

#-------------------------
# Archivos y colecciones
#-------------------------
nombre_base = "clase"
nombre_colecciones = ["usuarios", "destinos", "hoteles", "reservas", "actividades"]
nombre_archivos = ["usuarios.csv", "destinos.csv", "hoteles.csv", "reservas.csv", "actividades.csv"]

# -------------------------------------------------------------
# PROCESAMIENTO PRINCIPAL
# -------------------------------------------------------------
for nombre_archivo, nombre_coleccion in zip(nombre_archivos, nombre_colecciones):
    df = procesar_csv(nombre_archivo)
    if df is None:
        continue

    mongo.insertar_en_mongo(nombre_base, nombre_coleccion, df)
    redis.insertar_en_redis(nombre_coleccion, df)
    neo4j.crear_nodos_neo4j(nombre_coleccion, df)

    if nombre_coleccion == "reservas":
        neo4j.crear_relaciones_visito(df)

# -------------------------------------------------------------
# RELACIONES ENTRE USUARIOS
# -------------------------------------------------------------
neo4j.crear_relaciones_usuarios()

print("\n🚀 Proceso completado correctamente.")

✅ Se insertaron 60 documentos en 'usuarios'.
✅ Colección usuarios creada e insertada en MongoDB.
✅ Se registran 15 usuarios conectados en Redis.
✅ Nodos de Usuario creados exitosamente en Neo4j.
✅ Se insertaron 37 documentos en 'destinos'.
✅ Colección destinos creada e insertada en MongoDB.
✅ Nodos de Destino creados exitosamente en Neo4j.
✅ Se insertaron 117 documentos en 'hoteles'.
✅ Colección hoteles creada e insertada en MongoDB.
✅ Se insertaron 1685 documentos en 'reservas'.
✅ Colección reservas creada e insertada en MongoDB.
✅ Se insertaron 404 reservas temporales en Redis.
✅ Relaciones VISITO creadas exitosamente en Neo4j.
✅ Se insertaron 111 documentos en 'actividades'.
✅ Colección actividades creada e insertada en MongoDB.
✅ Relaciones entre usuarios creadas exitosamente en Neo4j.

🚀 Proceso completado correctamente.


## Verificación de la carga

Se generan bloques de consultas de prueba por cada una de las bases de datos para comprobar que contengan los datos cargados anteriormente.

### MongoDB

##### Cantidad de documentos existentes en cada colección

In [20]:
for i in range (len(nombre_colecciones)):
    cantidad=mongo.contar_documentos(nombre_base,nombre_colecciones[i])
    print(f"La cantidad de documentos de la coleccion {nombre_colecciones[i]} es {cantidad}")

La cantidad de documentos de la coleccion usuarios es 60
La cantidad de documentos de la coleccion destinos es 37
La cantidad de documentos de la coleccion hoteles es 117
La cantidad de documentos de la coleccion reservas es 1685
La cantidad de documentos de la coleccion actividades es 111


### Neo4j

In [21]:
df_nodos = neo4j.consulta(db_neo4j, """
    MATCH (n)
    RETURN labels(n)[0] AS tipo_nodo, COUNT(*) AS cantidad
""")

df_relaciones = neo4j.consulta(db_neo4j, """
    MATCH ()-[r]->()
    RETURN type(r) AS tipo_relacion, COUNT(*) AS cantidad
""")

print("Nodos por tipo:")
print(df_nodos.to_string(index=False))

print("\nRelaciones por tipo:")
print(df_relaciones.to_string(index=False))


Nodos por tipo:
tipo_nodo  cantidad
  Usuario        60
  Destino        37

Relaciones por tipo:
tipo_relacion  cantidad
       VISITO       661
  FAMILIAR_DE       118
     AMIGO_DE       478


### Redis

##### Cantidad y Vista Previa de Usuarios Conectados

In [22]:
claves = db_redis.keys("usuario:*:sesion")
print(f"Cantidad de usuarios conectados {len(claves)}\n")
print("Se imprimen los primeros 5:")
for clave in claves[:5]:
    usuario_id = clave.split(":")[1]
    estado = db_redis.get(clave)
    tiempo_restante = db_redis.ttl(clave)
    print(f"Usuario {usuario_id} → sesión: {estado} | TTL: {tiempo_restante} segundos")

Cantidad de usuarios conectados 15

Se imprimen los primeros 5:
Usuario 46 → sesión: activa | TTL: 3585 segundos
Usuario 53 → sesión: activa | TTL: 3585 segundos
Usuario 58 → sesión: activa | TTL: 3585 segundos
Usuario 1 → sesión: activa | TTL: 3585 segundos
Usuario 49 → sesión: activa | TTL: 3585 segundos


##### Cantidad y Vista Previa de Reservas en Carrito

In [23]:
claves = db_redis.keys("reserva_temp:*")
print(f"Cantidad de reservas temporales {len(claves)}\n")
print("Se imprimen las primeras 5:")
for clave in claves[:5]:
    datos = db_redis.hgetall(clave)
    tiempo_restante = db_redis.ttl(clave)
    print(f"{clave}: {datos} | TTL: {tiempo_restante} segundos")

Cantidad de reservas temporales 404

Se imprimen las primeras 5:
reserva_temp:1134: {'usuario_id': '17', 'destino_id': '22', 'fecha_reserva': '2025-09-28', 'precio_total': '170070'} | TTL: 3587 segundos
reserva_temp:310: {'usuario_id': '26', 'destino_id': '7', 'fecha_reserva': '2025-08-28', 'precio_total': '112826'} | TTL: 3587 segundos
reserva_temp:1781: {'usuario_id': '23', 'destino_id': '32', 'fecha_reserva': '2024-12-21', 'precio_total': '173348'} | TTL: 3587 segundos
reserva_temp:1957: {'usuario_id': '29', 'destino_id': '35', 'fecha_reserva': '2025-03-31', 'precio_total': '110803'} | TTL: 3587 segundos
reserva_temp:1771: {'usuario_id': '31', 'destino_id': '32', 'fecha_reserva': '2025-06-05', 'precio_total': '165349'} | TTL: 3587 segundos


## Generamos algunas búsquedas en caché

A partir de los datos cargados en MongoDB, se generan consultas ficticias para cargar un caché de búsqueda en Redis.

In [24]:
#Borrar redis
#db_redis.flushdb()


consultas = [
    ("hoteles", {"ciudad": "La Plata"}),
    ("destinos", {"tipo": "Playa", "ciudad": "La Rioja"}),
    ("hoteles", {"nombre": "Familia Galiano S.L. Hotel"}),
    ("actividades", {"tipo": "relax"})
]

for tipo, filtro in consultas:
    resultado = redis.obtener_cache(tipo, filtro)
    if resultado is None:
        cursor = mongo.obtener_cursor("clase",nombre_coleccion=tipo,filtro=filtro,proyeccion={"_id":0})
        resultado = list(cursor)
        if resultado:
            redis.guardar_en_cache(tipo,filtro,resultado,ttl=10)   

print("✅ Caché de búsqueda creado")

✅ Caché de búsqueda creado


### Vista del caché de búsqueda para lo guardado

A partir de las búsquedas cargadas anteriormente se puede ejecutar el siguiente bloque de código para comprobar que se hayan cargado. El tiempo de expiración seteado de ejemplo es de 10 segundos, por lo cual si se espera ese tiempo, se va a notar como las mismas se borran de Redis.

In [25]:
for tipo, filtro in consultas:
    resultados = redis.obtener_cache(tipo, filtro)
    if resultados:
        for resultado in resultados:
            print(resultado)
    else:
        print(f"No existe información en caché para la búsqueda {filtro} en {tipo}")

{'hotel_id': 1, 'nombre': 'Hermanos Soria S.Coop. Hotel', 'ciudad': 'La Plata', 'provincia': 'Buenos Aires', 'precio': 201178, 'calificacion': 4, 'servicios': ['restaurant', 'gimnasio', 'spa']}
{'hotel_id': 2, 'nombre': 'Espinosa & Asociados S.C.P Hotel', 'ciudad': 'La Plata', 'provincia': 'Buenos Aires', 'precio': 259467, 'calificacion': 3, 'servicios': ['spa', 'wifi']}
{'destino_id': 17, 'provincia': 'La Rioja', 'ciudad': 'La Rioja', 'pais': 'Argentina', 'tipo': 'Playa', 'precio_promedio': 138236}
{'hotel_id': 8, 'nombre': 'Familia Galiano S.L. Hotel', 'ciudad': 'San Fernando', 'provincia': 'Catamarca', 'precio': 103831, 'calificacion': 1, 'servicios': ['spa', 'restaurant']}
{'actividad_id': 2, 'nombre': 'Adaptive demand-driven framework', 'tipo': 'relax', 'ciudad': 'La Plata', 'provincia': 'Buenos Aires', 'precio': 38120}
{'actividad_id': 15, 'nombre': 'Sharable intangible frame', 'tipo': 'relax', 'ciudad': 'Resistencia', 'provincia': 'Chaco', 'precio': 26723}
{'actividad_id': 18, '